In [ ]:
# PPO 非常强大的结构， lr影响很大
from x_ml import PPO
import gymnasium as gym
import torch
ppo = PPO()
ppo.load_best_model()
env = gym.make("LunarLander-v3", render_mode="human")
obs, info = env.reset()
while True:
    action, _, _, _ = ppo.take_action(torch.tensor(obs).to(ppo.device).unsqueeze(0), deterministic=True)
    a = input("需要继续行动么？")
    if a.lower() == 'exit':
        break
    obs, reward, terminated, truncated, info = env.step(action.item())
    if terminated or truncated:
        break

In [ ]:
# advantage actor-critic LunarLander-v3
# # actor-critic 是真难train啊, CartPole-v1还能train,  LunarLander-v3 直接原地爆炸
# 几万个episode之后就稳定了
from x_ml import set_seed
from x_ml import ActorCritic

set_seed()

net = ActorCritic(
    env_id="LunarLander-v3",
    num_episodes=200000,
    actor_lr=5e-3,
    critic_lr=5e-3,
    gamma=0.99,
    eval_interval=2000,     # 对应你原来的 eval_internal（评估间隔）
    eval_episode=50,        # 对应你原来的 eval_eposide（评估局数）
    device="cuda:3",
    save_path="./model/best.pth",
    seed=0,                 # 可选：让环境也可复现
)

net.train()
net.inference(env_id="LunarLander-v3", render_mode="human")


In [ ]:
# advantage actor-critic CartPole-v1

from x_ml import set_seed
from x_ml import ActorCritic

set_seed()

net = ActorCritic(
    env_id="CartPole-v1",
    num_episodes=200000,
    actor_lr=5e-3,
    critic_lr=5e-3,
    gamma=0.99,
    eval_interval=2000,
    eval_episode=50,        
    device="cuda:3",
    save_path="./model/best.pth",
)

net.train()
net.inference(env_id="CartPole-v1", render_mode="human")


In [ ]:
# Dueling DQN for LunarLander-v3 真难收敛
# 增加net复杂度就好一点？实则未必 
# lr 从 1e-4 改 2e-3效果就好一点 估计加lr scheduling好不少 lr的重要性可以感受到？ 并没有更好，前期收敛快，但是之后没有稳定
# buffer 改大, lr 1e-3, num_eposide, target_update_step改大 这次收敛效果好不少 epsilon较大时 return_reward就比较稳定了, 平台期很长，但是eposide很大，之后也有提升
# num_eposide 改小试试, 测了一次, 感觉差不多, 但是感觉上大eposide能力强一点
import torch.nn as nn
from x_ml import set_seed, DDDQN
import gymnasium as gym
set_seed()
class Q_net(nn.Module):
    def __init__(self, input_dim=8, output_dim=4):
        super().__init__()
        self.layer1 = nn.Linear(input_dim, 96)
        self.layerV = nn.Linear(96, 1)
        self.layerA = nn.Linear(96, output_dim)

        self.activation = nn.ReLU()
    
    def forward(self, x):
        x = self.layer1(x)
        x = self.activation(x)

        v = self.layerV(x)
        a = self.layerA(x)
        
        q = v + a - a.mean(dim=1, keepdim=True)
        return q


dqn = DDDQN(net=Q_net, env=gym.make("LunarLander-v3"), eval_env=gym.make("LunarLander-v3"), lr=1e-3,target_update_step=1000,
          num_episodes=600, epsilon_end=0.05, eval_episode=50, eval_step=50, buffer_size=100000, gamma=0.99, minimal_size=2000, device="cuda",batch_size=128,criterion=nn.SmoothL1Loss)
dqn.train()

dqn.inference(gym.make("LunarLander-v3", render_mode="human"))

In [ ]:
# DDQN for LunarLander-v3
from x_ml import set_seed, DDQN
import gymnasium as gym
import torch.nn as nn
set_seed()

class Qnet(nn.Module):
    def __init__(self, input_dim=8, output_dim=4):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 96),
            nn.ReLU(),
            nn.Linear(96, 192),
            nn.ReLU(),
            nn.Linear(192, 96),
            nn.ReLU(),
            nn.Linear(96, output_dim)
        )
    
    
    def forward(self, x):
        x = self.net(x)
        return x


dqn = DDQN(net=Qnet, env=gym.make("LunarLander-v3"), eval_env=gym.make("LunarLander-v3"), lr=1e-4,target_update_step=100,
          num_episodes=600, epsilon_end=0.05, eval_episode=50, eval_step=50, buffer_size=10000, gamma=0.99, minimal_size=200, device="cuda",batch_size=128,criterion=nn.SmoothL1Loss)

dqn.train()
dqn.inference(gym.make("LunarLander-v3", render_mode="human"))

In [ ]:
## DQN for LunarLander-v3 eposide 300后开始大幅提升

from x_ml.x_dqn import DQN
import torch.nn as nn
from x_ml import set_seed
import gymnasium as gym
set_seed()

class Qnet(nn.Module):
    def __init__(self, input_dim=8, output_dim=4):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 96),
            nn.ReLU(),
            nn.Linear(96, 192),
            nn.ReLU(),
            nn.Linear(192, 96),
            nn.ReLU(),
            nn.Linear(96, output_dim)
        )
    
    
    def forward(self, x):
        x = self.net(x)
        return x


dqn = DQN(action_dim=4,net=Qnet, env=gym.make("LunarLander-v3"), eval_env=gym.make("LunarLander-v3"), lr=1e-4,target_update_step=100,
          num_episodes=600, epsilon_end=0.05, eval_episode=50, eval_step=100, buffer_size=10000, gamma=0.99, minimal_size=200, device="cuda",batch_size=128,criterion=nn.SmoothL1Loss)

dqn.train()
dqn.inference(gym.make("LunarLander-v3", render_mode="human"))

In [ ]:
# DQN for CartPole-v1
# lr 1e-4 train 不动， 2e-3嘎嘎猛

from x_ml import DQN
from x_ml import set_seed
set_seed()
dqn = DQN(action_dim=2, lr=2e-3)
dqn.train()
dqn.inference()